In [67]:
import sys
sys.path.append("../python")

from importlib import reload
import json
import h5py
import numpy as np
import arviz as az
import compute_costs
import evaluations
import matplotlib.pyplot as plt
import matplotlib.patches as pt
import math
from prettytable import PrettyTable

from matplotlib.colors import ListedColormap
cmap_blue = ListedColormap(["white", "lightsteelblue", "cornflowerblue", "royalblue", "blue", "navy"])
cmap = ListedColormap(["white", "darkorange", "gold", "navy", "steelblue", "yellowgreen"])
cmap_light = ListedColormap(["white", "bisque", "gold", "lavender", "aliceblue", "honeydew"])
cmap_gray = ListedColormap(["lightgray"])

In [68]:
### MH ###
'''parameters_paths = [
    "../configs/test_mh/mh_test_2_low_variance.json",
    "../configs/test_mh/mh_test_1.json",
    "../configs/test_mh/mh_test_2_high_variance.json"
]

result_paths = [
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_2_low_variance",
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_1",
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_2_high_variance"
] 

settings = ["MH (low prop. variance)", "MH", "MH (high prop. variance)"]
'''

'parameters_paths = [\n    "../configs/test_mh/mh_test_2_low_variance.json",\n    "../configs/test_mh/mh_test_1.json",\n    "../configs/test_mh/mh_test_2_high_variance.json"\n]\n\nresult_paths = [\n    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_2_low_variance",\n    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_1",\n    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_2_high_variance"\n] \n\nsettings = ["MH (low prop. variance)", "MH", "MH (high prop. variance)"]\n'

In [69]:
### MH and MLDA ###
parameters_paths = [
    "../configs/test_mh/mh_test_1.json",
    "../configs/test_mlda/mlda_test_1.json",
    "../configs/test_mlda/mlda_test_2.json",
    "../configs/test_mlda/mlda_test_3.json"
]

result_paths = [
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mh/mh_test_1",
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mlda/mlda_test_1",
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mlda/mlda_test_2",
    "/home/anne/Masterarbeit/masterarbeit/2d/results/test_mlda/mlda_test_3"
] 

settings = ["MH", "MLDA 1", "MLDA 2", "MLDA 3"]

extra_burn_in = [0, 0, 10000, 0]


In [70]:
num_samples = []
num_burn_in = []
for path in parameters_paths:
    file = open(path)
    config = json.load(file)
    file.close()
    num_samples.append(config["Sampling"]["NumSamples"])
    num_burn_in.append(config["Sampling"]["BurnIn"])

# load samples
number_of_chains = config["Setup"]["Chains"]
number_of_experiments = len(result_paths)

samples = []
i = 0
for path in result_paths:
    result_list = []
    for c in range(number_of_chains):
        fin = h5py.File(path + "_" + str(c) + ".h5")
        result_list.append(np.array(fin['/samples'][:,extra_burn_in[i]:]))
    fin.close()
    samples.append(result_list)
    i += 1

In [71]:
costs = np.zeros(number_of_experiments)
for i in range(number_of_experiments):
    costs[i] = evaluations.costs_per_sample(parameters_paths[i])

Dipole:
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
volume_conductor.time_creating_grid = 0.589178
volume_conductor.time_reordering_labels = 0.0034385
volume_conductor.time = 0.592619
time_element_neighborhood_map 0.822447 s
degree = 1
element_type = hexahedron
Dipole[position: [120 220] moment: [0.0707372 0.997495]]
sigma = 1.1508410054784861
{'Level3_V': {0: array([-6.81636379, -9.01600192, -7.71058625, -8.35094731, -8.73544543,
       -8.72777265, -8.96513867, -8.83761558, -6.49859219, -9.45857159,
       -6.73752393, -9.55589385, -6.84710488, -8.46125499, -8.11286581,
       -7.54598507, -9.77744188, -5.43484199, -5.45710847, -4.64881048,
       -0.95582971,  4.09169358, 14.02398741, 18.19214849, 21.79231406,
       22.05741967, 20.16578684, 18.87710819, 15.59932767,  9.87505027,
       10.08822778,  2.75373647,  2.10600598,  2.36298441,  0.05759877,
        0.35689352])}}
volume_conductor.time_creating_grid = 4.03165
volume_conductor.time_reordering_labels = 0.866831


In [72]:
ess = np.zeros((number_of_experiments,number_of_chains))
samples_per_es = np.zeros((number_of_experiments,number_of_chains))
costs_per_es = np.zeros((number_of_experiments,number_of_chains))
costs_per_1kes = np.zeros((number_of_experiments,number_of_chains))

for i in range(number_of_experiments):
    for j in range(number_of_chains):
        chain = samples[i][j]
        ess[i,j] = np.around(min(az.ess(chain[0]),az.ess(chain[1]),az.ess(chain[2])),2)
        samples_per_es[i,j] =  np.around((num_samples[i]-num_burn_in[i]-extra_burn_in[i])/ess[i,j],2)
        costs_per_es[i,j] = np.around(costs[i]*samples_per_es[i,j],2)
        costs_per_1kes[i,j] = int(1000*costs_per_es[i,j] + costs[i]*(num_burn_in[i]+extra_burn_in[i]))

In [73]:
for j in range(number_of_chains):
    point = (config["ModelConfig"]["Dipoles"][j][0],config["ModelConfig"]["Dipoles"][j][1],config["ModelConfig"]["Dipoles"][j][2])

    for i in range(number_of_experiments):
        if not evaluations.check_results(samples[i][j], point, tol=[5,5,0.25]):
            print("There is a problem with chain %i of experiment %s" % (j, settings[i]))
        if not evaluations.burn_in_check(samples[i][j]):
            print("Burn in check failed for chain %i of experiment %s" % (j, settings[i]))
        '''elif not evaluations.is_converged(samples[i][j]):
            print("Chain %i of experiment %s is not converged." % (j, settings[i]))'''

Burn in check failed for chain 0 of experiment MH
There is a problem with chain 0 of experiment MLDA 1
Burn in check failed for chain 0 of experiment MLDA 1
There is a problem with chain 1 of experiment MLDA 1
Burn in check failed for chain 1 of experiment MLDA 1
There is a problem with chain 1 of experiment MLDA 2
Burn in check failed for chain 1 of experiment MLDA 2
There is a problem with chain 2 of experiment MH
There is a problem with chain 2 of experiment MLDA 2
There is a problem with chain 3 of experiment MH
Burn in check failed for chain 3 of experiment MH
Burn in check failed for chain 3 of experiment MLDA 1
There is a problem with chain 3 of experiment MLDA 2
There is a problem with chain 5 of experiment MLDA 1
There is a problem with chain 7 of experiment MLDA 1
There is a problem with chain 7 of experiment MLDA 2
Burn in check failed for chain 7 of experiment MLDA 2
There is a problem with chain 9 of experiment MH
There is a problem with chain 9 of experiment MLDA 1
Burn i

In [74]:
t = PrettyTable(['Algorithm', 'ESS', 'Samples per ES', 'Costs per sample (s)', 'Costs per ES (s)', 'Costs of 1K ES (s) with Burn In'])

eval_chains = [0,1,2,3,4]

for j in eval_chains:
    for i in range(number_of_experiments):
        t.add_row([settings[i],ess[i,j],samples_per_es[i,j],np.around(costs[i],3),costs_per_es[i,j],costs_per_1kes[i,j]])
    t.add_row(["","","","","",""])
print(t)

+-----------+--------+----------------+----------------------+------------------+---------------------------------+
| Algorithm |  ESS   | Samples per ES | Costs per sample (s) | Costs per ES (s) | Costs of 1K ES (s) with Burn In |
+-----------+--------+----------------+----------------------+------------------+---------------------------------+
|     MH    | 12.92  |    15479.88    |        0.005         |      82.48       |             82852.0             |
|   MLDA 1  |  2.23  |    40358.74    |        0.025         |      992.94      |             993186.0            |
|   MLDA 2  | 97.69  |     818.92     |        0.048         |      39.27       |             40229.0             |
|           |        |                |                      |                  |                                 |
|     MH    | 248.36 |     805.28     |        0.005         |       4.29       |              4662.0             |
|   MLDA 1  |  6.28  |    14331.21    |        0.025         |      352.

In [75]:
t = PrettyTable(['Algorithm', 'ESS', 'Samples per ES', 'Costs per sample (s)', 'Costs per ES (s)', 'Costs of 1K ES (s) with Burn In'])

eval_chains = [5,6,7,8,9]

for j in eval_chains:
    for i in range(number_of_experiments):
        t.add_row([settings[i],ess[i,j],samples_per_es[i,j],np.around(costs[i],3),costs_per_es[i,j],costs_per_1kes[i,j]])
    t.add_row(["","","","","",""])
print(t)

+-----------+---------+----------------+----------------------+------------------+---------------------------------+
| Algorithm |   ESS   | Samples per ES | Costs per sample (s) | Costs per ES (s) | Costs of 1K ES (s) with Burn In |
+-----------+---------+----------------+----------------------+------------------+---------------------------------+
|     MH    |  404.42 |     494.54     |        0.005         |       2.64       |              3012.0             |
|   MLDA 1  |   21.9  |    4109.59     |        0.025         |      101.11      |             101356.0            |
|   MLDA 2  |  63.96  |    1250.78     |        0.048         |      59.98       |             60939.0             |
|           |         |                |                      |                  |                                 |
|     MH    | 1365.28 |     146.49     |        0.005         |       0.78       |              1152.0             |
|   MLDA 1  | 1282.25 |     70.19      |        0.025         | 